In [78]:
from PIL import Image
import numpy as np
import os
import pandas as pd
import cv2

from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow

from keras.initializers import glorot_uniform
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D


In [69]:
DEBUG = False

def debug(str):
    if DEBUG:
        print(str)

In [70]:
"""
Reference:
https://stackoverflow.com/questions/25008458/how-to-apply-clahe-on-rgb-color-images

CLAHE algorithm cannot be applied directly on RGB images

Implementation of CLAHE algorithm on a single image
img_path = "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Catla/Body/Catla-Body (1).JPG"
img = cv2.imread(img_path, cv2.IMREAD_COLOR)
img = cv2.resize(img,(256,256))
img = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)

#configure CLAHE
clahe = cv2.createCLAHE(clipLimit=10)

#0 to 'L' channel, 1 to 'a' channel, and 2 to 'b' channel
img[:,:,0] = clahe.apply(img[:,:,0])

img = cv2.cvtColor(img, cv2.COLOR_Lab2RGB)

cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()
"""

image_array = []
store_image = []
store_labels = []
#train_path='/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Catla/Body'
#train_images=os.listdir('/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Catla/Body')

def store_inarray(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (512, 512))
    image = cv2.cvtColor(image, cv2.COLOR_RGB2Lab)
    clahe = cv2.createCLAHE(clipLimit=10)
    image[:,:,0] = clahe.apply(image[:,:,0])
    image = cv2.cvtColor(image, cv2.COLOR_Lab2RGB)
    image=np.array(image)
    store_image.append(image)

In [71]:
directories_path = "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak"
directories = os.listdir("/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak")
directories = np.sort(directories)
__train_path_array__ = []
for directory in directories:
    __train_path_array__.append(directories_path +'/' + directory + '/Body')
    __train_path_array__.append(directories_path +'/' + directory + '/Head')
    __train_path_array__.append(directories_path +'/' + directory + '/Scales')

for train_path in __train_path_array__:
    if ((train_path == "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Catla/Body") or(train_path == "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Catla/Head") or (train_path == "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Catla/Scales")):
        images=os.listdir(train_path)
        for image in images:
            image_array.append((image, str(train_path + 'image')))
            store_labels.append(0)
            image_path=train_path + '/' + image
            debug(image_path)
            store_inarray(image_path)
    elif ((train_path == "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Cyprinus carpio/Body") or (train_path == "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Cyprinus carpio/Head") or (train_path == "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Cyprinus carpio/Scales")):
        images=os.listdir(train_path)
        for image in images:
            image_array.append((image, str(train_path + 'image')))
            store_labels.append(1)
            image_path=train_path + '/' + image
            debug(image_path)
            store_inarray(image_path)
    elif (train_path == "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Grass Carp/Body" or "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Grass Carp/Head" or "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Grass Carp/Scales"):
        images=os.listdir(train_path)
        for image in images:
            image_array.append((image, str(train_path + 'image')))
            store_labels.append(2)
            image_path=train_path + '/' + image
            debug(image_path)
            store_inarray(image_path)
    elif ((train_path == "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Mori/Body") or (train_path == "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Mori/Head") or (train_path == "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Mori/Scales")):
        images=os.listdir(train_path)
        for image in images:
            image_array.append((image, str(train_path + 'image')))
            store_labels.append(3)
            image_path=train_path + '/' + image
            debug(image_path)
            store_inarray(image_path)
    elif ((train_path == "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Silver/Body") or (train_path == "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Silver/Head") or (train_path == "/home/adarsh/Downloads/Machine learning/Marine species classification/Fish-Pak/Silver/Scales"):
        images=os.listdir(train_path)
        for image in images:
            image_array.append((image, str(train_path + 'image')))
            store_labels.append(4)
            image_path=train_path + '/' + image
            debug(image_path)
            store_inarray(image_path)

In [74]:
# Build a dataframe        
fishes_df = pd.DataFrame(data=image_array, columns=['Fish type', 'image'])
print(fishes_df.head())
print(fishes_df.tail())

             Fish type                                              image
0  Catla-Body (15).JPG  /home/adarsh/Downloads/Machine learning/Marine...
1  Catla-Body (20).JPG  /home/adarsh/Downloads/Machine learning/Marine...
2  Catla-Body (10).JPG  /home/adarsh/Downloads/Machine learning/Marine...
3   Catla-Body (6).JPG  /home/adarsh/Downloads/Machine learning/Marine...
4   Catla-Body (9).JPG  /home/adarsh/Downloads/Machine learning/Marine...
                 Fish type                                              image
910   Silver-Scale (7).JPG  /home/adarsh/Downloads/Machine learning/Marine...
911  Silver-Scale (50).JPG  /home/adarsh/Downloads/Machine learning/Marine...
912  Silver-Scale (12).JPG  /home/adarsh/Downloads/Machine learning/Marine...
913   Silver-Scale (4).JPG  /home/adarsh/Downloads/Machine learning/Marine...
914  Silver-Scale (35).JPG  /home/adarsh/Downloads/Machine learning/Marine...


In [75]:
np.save('/home/adarsh/Downloads/Machine learning/Marine species classification/data.npy',store_image)
np.save('/home/adarsh/Downloads/Machine learning/Marine species classification/labels.npy',store_labels)

In [76]:
training_data = np.load("/home/adarsh/Downloads/Machine learning/Marine species classification/data.npy")
training_labels = np.load("/home/adarsh/Downloads/Machine learning/Marine species classification/labels.npy")


In [77]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


training_data, training_labels = shuffle(training_data, training_labels, random_state=1)

train_x, test_x, train_y, test_y = train_test_split(training_data, training_labels, test_size=0.05, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(869, 512, 512, 3)
(869,)
(46, 512, 512, 3)
(46,)
